In [ ]:
from fair import FAIR

In [ ]:
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import copy

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
from tqdm.auto import tqdm

In [ ]:
from scipy.stats import linregress

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1750, 2101, 1)

In [ ]:
scen_df = pd.read_csv('../data/c1_emissions_scenarios.csv')

In [ ]:
scen_labels = pd.unique(list(zip(scen_df.Model, scen_df.Scenario)))

In [ ]:
scenarios = [f'{scen[0]}___{scen[1]}' for scen in scen_labels]

In [ ]:
# do one at a time
#scenarios = [scenarios[0]]
f.define_scenarios(scenarios)
fair_params_df = pd.read_csv('../data/ar6_calibration_ebm3.csv', index_col=0)

In [ ]:
f.define_configs(list(fair_params_df.index))

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# declare species and properties
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
hist_df = pd.read_csv('../data/historical_emissions.csv')

In [ ]:
hist_df
pd.unique(scen_df['Variable'])

In [ ]:
for scenario in scenarios:
    f.emissions.loc[
        dict(scenario=scenario, specie='CO2', timepoints=np.arange(1750.5, 2015))
    ] = hist_df.loc[hist_df['Variable'].str.startswith("Emissions|CO2|"), '1750':'2014'].values.sum(axis=0, keepdims=True).T * 1/1000
    f.emissions.loc[dict(scenario=scenario, specie='CH4')] = 0
    f.emissions.loc[dict(scenario=scenario, specie='N2O')] = 0
    f.concentration.loc[dict(scenario=scenario, specie='CH4')] = 729.2
    f.concentration.loc[dict(scenario=scenario, specie='N2O')] = 270.1
    
    model, scen = scenario.split('___')
    f.emissions.loc[
        dict(scenario=scenario, specie='CO2', timepoints=np.arange(2015.5, 2101))
    ] = scen_df.loc[
        (scen_df['Variable'].str.startswith("AR6 climate diagnostics|Infilled|Emissions|CO2|")) &
        (scen_df['Scenario']==scen) &
        (scen_df['Model']==model),
        '2015':'2100'
    ].values.sum(axis=0, keepdims=True).T * 1/1000

In [ ]:
f.emissions

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'gamma'])
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'rT'].values.squeeze(), specie='CO2')

# Scale CO2 forcing based on its 4xCO2 calibration
calibrated_f4co2_mean = fair_params_df.loc[:,'F_4xCO2'].values.mean()
fill(
    f.species_configs['forcing_scale'], 
    1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[:,'F_4xCO2'].values)/calibrated_f4co2_mean,
    specie='CO2'
)

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs['baseline_concentration'], 
    fair_params_df.loc[:, 'co2_concentration_1750'].values.squeeze(),
    specie='CO2'
)

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
f.forcing

In [ ]:
pl.plot(f.temperature[:,96,:,0]);

In [ ]:
pl.plot(f.emissions[:, 96, 0, 0])

In [ ]:
pl.plot(f.concentration[:,96,:,0]);

## Find year of net zero CO2

In [ ]:
first_negative = np.array([f.emissions[:,:,0,0]<0]).argmax(axis=1).squeeze()
first_negative

In [ ]:
#f.emissions[-5:,9,0,0]
# need to exclude scenarios 2, 7 and 9
f.cumulative_emissions[first_negative, :, 0, 0]

In [ ]:
netzero_to_2100_cumulative_emissions = np.zeros(97)
for iscen in tqdm(range(97)):
    netzero_to_2100_cumulative_emissions[iscen] = (f.cumulative_emissions[-1,iscen,0,0] - f.cumulative_emissions[first_negative[iscen], iscen, 0, 0])
#f.cumulative_emissions[-2:,iscen,0,0].mean(axis=0) - f.cumulative_emissions[first_negative[iscen], iscen, 0, 0]

In [ ]:
# exclude scenarios that don't reach net zero
netzero_to_2100_cumulative_emissions[netzero_to_2100_cumulative_emissions > 0] = np.nan
netzero_to_2100_cumulative_emissions

In [ ]:
peak_times = np.zeros((97, 1001))
#for iscen in range(97):
#    for iconfig in range(1001):
#        peak_times[iscen, iconfig] = np.array([f.temperature[:,iscen,iconfig,0]]).argmax()
peak_times = f.temperature[:,:,:,0].argmax(axis=0)

In [ ]:
#pl.hist(peak_times[2,:])

In [ ]:
temperature_drawdown = np.zeros((97,1001))
for iscen in tqdm(range(97)):
    for iconfig in range(1001):
        temperature_drawdown[iscen, iconfig] = f.temperature[int(peak_times[iscen, iconfig]),iscen,iconfig,0]-f.temperature[-1,iscen,iconfig,0]

In [ ]:
pl.hist(temperature_drawdown[2,:])

In [ ]:
pc05 = np.percentile(temperature_drawdown, 5, axis=1)
pc50 = np.percentile(temperature_drawdown, 50, axis=1)
pc95 = np.percentile(temperature_drawdown, 95, axis=1)

netzero = ~np.isnan(netzero_to_2100_cumulative_emissions)

In [ ]:
netzero_to_2100_cumulative_emissions[netzero]

In [ ]:
netzero & (pc05 > 0)

In [ ]:
pl.scatter(-pc50[netzero & (pc50>0)], netzero_to_2100_cumulative_emissions[netzero & (pc50>0)])
pl.scatter(-pc05[netzero & (pc05>0)], netzero_to_2100_cumulative_emissions[netzero & (pc05>0)])
pl.scatter(-pc95[netzero & (pc95>0)], netzero_to_2100_cumulative_emissions[netzero & (pc95>0)])

sl, ic, _, _, _ = linregress(-pc50[netzero & (pc50>0)], netzero_to_2100_cumulative_emissions[netzero & (pc50>0)])
x = np.linspace(-0.35, 0)
pl.plot(x, sl*x + ic)
print(sl, ic)

sl, ic, _, _, _ = linregress(-pc05[netzero & (pc05>0)], netzero_to_2100_cumulative_emissions[netzero & (pc05>0)])
x = np.linspace(-0.2, 0)
pl.plot(x, sl*x + ic)
print(sl, ic)

sl, ic, _, _, _ = linregress(-pc95[netzero & (pc95>0)], netzero_to_2100_cumulative_emissions[netzero & (pc95>0)])
x = np.linspace(-0.5, 0)
pl.plot(x, sl*x + ic)
print(sl, ic)

In [ ]:
netzero_to_2100_cumulative_emissions[~np.isnan(netzero_to_2100_cumulative_emissions)]

In [ ]:
sl, ic, _, _, _ = linregress(-np.percentile(temperature_drawdown[~np.isnan(netzero_to_2100_cumulative_emissions), :], 95, axis=1), netzero_to_2100_cumulative_emissions[~np.isnan(netzero_to_2100_cumulative_emissions)])
x = np.linspace(-0.35, 0)
pl.plot(x, sl*x + ic)
print(sl, ic)

In [ ]:
pc05 = np.percentile(temperature_drawdown, 5, axis=1)
pc05

In [ ]:
netzero_to_2100_cumulative_emissions[~np.isnan(netzero_to_2100_cumulative_emissions)&(pc05>0)]

In [ ]:
sl, ic, _, _, _ = linregress(
    -pc05[~np.isnan(netzero_to_2100_cumulative_emissions)&(pc05>0)], 
    netzero_to_2100_cumulative_emissions[~np.isnan(netzero_to_2100_cumulative_emissions)&(pc05>0)]
)
print(sl, ic)